## Форматирование и обогащение данных
### Форматирование, удобное для изучения и машинного обучения
#### Загрузка библиотек

In [ ]:
import numpy  as np
import pandas as pd

In [ ]:
# Настройка отображения данных в Jupyter notebook
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows', 100)
pd.set_option('precision', 3)

In [ ]:
# Visualization and Graphics
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt
# !conda install seaborn 
import seaborn as sns
plt.rcParams['figure.figsize'] = (7,7)   # (8,6)

#!pip install ggplot
matplotlib.style.use('ggplot')       # Use ggplot style plots

## Обработка данных
### Первичная очистка
Первичная очистка и форматирование данных сделаны с помощью preprocess_data.R:
<br> [`preprocess_data.R`](https://github.com/AlexanderArtemyev/Wilson-disease/blob/master/preprocess_data.R) в кодировке cp1251. 
<br> [`preprocess_data.utf8.R`](https://github.com/AlexanderArtemyev/Wilson-disease/blob/master/preprocess_data.utf8.R) в кодировке utf-8.

### Чтение очищеных данных

In [ ]:
data_dir = '../data_transforemed/'

df = pd.read_csv(data_dir + 'Wilson_anonym.csv', sep=';', encoding='cp1251')
df.head(5).tail(3)

### Группировка и комбинирование признаков
#### Группы признаков
Группы выделены в соответствии со смыслом данных, логикой их использования и обработки, форматом представления:
- `target`
- `relatives`
- `sex`, `sex_cat`
- `bmi`, `bmi_scaled`
- `symptom`
- `cirrhosis`
    - `childpugh_dummy`
- `activity`
- `debut_age`, `debut_age_scaled`
- `debut_organ`
- `genetic`
- `genetic_dummy`
- `genetic__1`, `genetic__2`
- `genetic_risk__1`, `genetic_risk__2`
- `genetic_risk__1_scaled`, `genetic_risk__2_scaled`
- `data`
- `exclude`, `exclude_model`

<br> Вспомогательные признаки
- `num_to_scale`
- `num_scaled`
- `genetic_risk`
- `genetic_risk_scaled`

In [ ]:
target        = ['TargetHead']                                               # TargetHead = ifelse(Target == 3, 1, 0)
relatives     = ['TargetHeadRelativeMax']                                    # Наихудший диагноз у родсттвенников: 0, 1, NaN
sex           = ['Sex']                                                      # Пол
bmi           = ['BMI', 'Height', 'Mass']                                    # Идекс массы тела, Рост, Вес
symptom       = ['KKF']                                                      # Симптом: Кольца Кайзера — Флейшера
cirrhosis     = ['Cirrhosis', 'ChildPugh', 'Advanced']                       # Цирроз: cтадия
activity      = ['Activity']                                                 # Скорость прогрессирования (активность) цирроза
debut_age     = ['DebutAge']                                                 # Дебют заболвания: возраст
debut_organ   = ['DebutLiver', 'DebutNeuro', 'DebutKidney', 'DebutEndocr',   # Дебют заболвания: что выявили
                          'DebutSibs', 'DebutVasku', 'DebutGemAnem', 'DebutSelez', 'DebutOther']
genetic       = ['F2', 'F5', 'F7', 'F13', 'ITGA2', 'ITGB3',                  # Генетические признаки 
                          'PAI_1', 'FGB', 'MTHFR_677', 'MTHFR_1298']
exclude       = ['Target', 'TargetRelativeMax']                              # Исключить из данных
exclude_model = ['FamilyID', 'DebutNeuro'] + exclude                         # Исключить из модели

In [ ]:
features = [
    ('target',          target),
    ('relatives',       relatives),
    ('sex',             sex),
    ('bmi',             bmi),
    ('symptom',         symptom),
    ('cirrhosis',       cirrhosis),
    ('activity',        activity),
    ('debut_age',       debut_age),
    ('debut_organ',     debut_organ),
    ('genetic',         genetic),
    ('exclude',         exclude),
    ('exclude_model',   exclude_model)
]

#### Комбинирование признаков: объединение и исключение групп

In [ ]:
def combine_features(combined_features_groups,
                     exclude_features):
    '''all_features_list - объединить эти группы признаков
       exclude_features  - исключить эти признаки
    '''
    result = list()
    for sublist in combined_features_groups:
        for item in sublist:
            result.append(item)
    result = [x for x in result if x not in exclude_features]
    return(result)

### Приведение данных к виду, удобному для изучения и машинного обучения
1. Преобразование и масштабирование числовых признаков
2. Формат пола - числовой и категориальный: М, F
3. Обогащение, группировка, масштабирование генетических признаков

#### Преобразование и масштабирование числовых признаков

In [ ]:
df['TargetHeadRelativeMax'] = np.nan                                  # Нет данных о родственниках
df.loc[(df['TargetRelativeMax'] == 1, 'TargetHeadRelativeMax')] = 0   # Нет осложнений у родственников
df.loc[(df['TargetRelativeMax'] == 3, 'TargetHeadRelativeMax')] = 1   # Есть осложнения у родственников
#df.loc[list(df['TargetRelativeMax'] != 3)]

In [ ]:
df['BMI_scaled']       = (df['BMI'].copy() - 23) / 4
df['Height_scaled']    = (df['Height'].copy() - 1.75) / 0.1
df['Mass_scaled']      = (df['Mass'].copy() - 70) / 15
df['DebutAge_scaled']  = (df['DebutAge'].copy() - 18) / 8
df['ChildPugh_scaled'] = (df['ChildPugh'] - 1.43) / 1.

#### Группы признаков
При выборе признаков хотим объединять их в группы

In [ ]:
num_to_scale     = ['BMI','Height','Mass','DebutAge','ChildPugh']
bmi_scaled       = ['BMI_scaled','Height_scaled','Mass_scaled']
debut_age_scaled = ['DebutAge_scaled']
num_scaled       = bmi_scaled + debut_age_scaled + ['ChildPugh_scaled']
#num_scaled       = ['BMI_scaled','Height_scaled','Mass_scaled','DebutAge_scaled']

In [ ]:
features = features + [('num_to_scale',     num_to_scale)]
features = features + [('num_scaled',       num_scaled)]
features = features + [('bmi_scaled',       bmi_scaled)]
features = features + [('debut_age_scaled', debut_age_scaled)]

In [ ]:
print( df[num_to_scale].describe() )
print( df[num_scaled].describe() )

#### Формат пола: M, F

In [ ]:
def convert_sex(x):
    if x == 1:
        return 'M'
    else:
        return 'F'

df['Sex M/F'] = df.Sex.apply(func=convert_sex)

In [ ]:
sex_cat = ['Sex M/F']

features = features + [('sex_cat', sex_cat)]

### Формирование таблицы `df_ext` для feature engineering

In [ ]:
data_features_ext = combine_features(
                       [target, relatives, 
                        sex, sex_cat, 
                        bmi, bmi_scaled,
                        symptom, 
                        cirrhosis, activity,
                        debut_age, debut_age_scaled,
                        debut_organ,
                        genetic ],
                       exclude)

print(data_features_ext)

df_ext = df[data_features_ext]
df_ext.head(4)

### Группа `cirrhosis`
#### Dummy encoding для 'ChildPugh' 

In [ ]:
df_dummies = pd.get_dummies(df[['ChildPugh']], prefix=None, prefix_sep='_', dummy_na=False, 
                      columns=['ChildPugh'], sparse=False, drop_first=False)

df_ext = pd.concat([df_ext, df_dummies], axis = 1)   # https://pandas.pydata.org/pandas-docs/stable/merging.html

childpugh_dummy = list( df_dummies.columns )

features = features + [('childpugh_dummy', childpugh_dummy)]

print(df_dummies.head(3))

df_ext.head(3)

### Генетические признаки
#### Обогащение, группировка, масштабирование генетических признаков

##### Dummy encoding для генетических признаков.

In [ ]:
df_dummies = pd.get_dummies(df[genetic], prefix=None, prefix_sep='_', dummy_na=False, 
                       columns=genetic, sparse=False, drop_first=False)

df_ext = pd.concat([df_ext, df_dummies], axis = 1)   # https://pandas.pydata.org/pandas-docs/stable/merging.html
df_dummies.head(3)

In [ ]:
genetic_dummy = list( df_dummies.columns )

print(genetic)
print(genetic_dummy)

In [ ]:
features = features + [('genetic_dummy', genetic_dummy)]

##### Формальное отображение каждого из генетических признаков в массив [0,1,2]
Наиболее частый гомозинготный аллельный вариант: 0,
<br> гетерозиготный вариант: 1,
<br> редкий гомозинготный аллельный вариант: 2.

In [ ]:
gen_map_1 = [
          {"F2":         {"GG":0,   "GA":1,     "AA":2}},
          {"F5":         {"GG":0,   "GA":1,     "AA":2}},
          {"F7":         {"GG":0,   "GA":1,     "AA":2}},
          {"F13":        {"GG":0,   "GT":1,     "TT":2}},
          {"ITGA2":      {"CC":0,   "CT":1,     "TT":2}},
          {"ITGB3":      {"TT":0,   "TC":1,     "CC":2}},
          {"PAI_1":      {"5G5G":0, "5G4G":1, "4G4G":2}},
          {"FGB":        {"GG":0,   "GA":1,     "AA":2}},
          {"MTHFR_677":  {"CC":0,   "CT":1,     "TT":2}},
          {"MTHFR_1298": {"AA":0,   "AC":1,     "CC":2}}

    ]

df_1 = pd.DataFrame()
for m in gen_map_1:    
    [(k,v)] = m.items()
    #print(k,v)
    df_1[k + '__1'] = df[k].map(v)

df_ext = pd.concat([df_ext, df_1], axis = 1)   # https://pandas.pydata.org/pandas-docs/stable/merging.html

df_1.head(3)

In [ ]:
genetic__1 = list( df_1.columns )
print(genetic__1)

features = features + [('genetic__1',     genetic__1)]

##### Отображение каждого из генетических признаков  в массив [0,1,2] <br> с помощью открытых баз знаний медицинских лабораторий

Использованы открытые базы знаний лабораторий: helix.ru, invitro.ru
<br> В базах знаний описана связь гомозиготной комбинации нуклеотидов с увеличением или снижением риска тромбофилии:

Указано, что **гомозиготная** комбинация влияет на риск тромбофилии: $\pm 2$. 
<br> Указано, что **гетерозиготная** комбинация влияет на риск тромбофилии: $\pm 1$. Это не всегда понятно из описания.
<br> Аллельные варианты нуклеотидов (гетерозиготный, гомозиготный) **не связанные** с риском тромбофилии: $0$. 


In [ ]:
# Предварительная попытка использовать базу знаний
# Эту ячейку собираюсь удлаить
gen_map_2 = [
          {"F2":          {"GG":0,   "GA":1,   "AA":2}},
          {"F5":          {"GG":0,   "GA":1,   "AA":2}},
          {"F7":          {"GG":0,   "GA":-1,  "AA":-2}},
          {"F13":         {"GG":0,   "GT":0,   "TT":-1}},
          {"ITGA2":       {"CC":0,   "CT":0,   "TT":1}},
          {"ITGB3":       {"TT":0,   "TC":1,   "CC":2}},
          {"PAI_1":       {"5G5G":0, "5G4G":0, "4G4G":1}},
          {"FGB":         {"GG":0,   "GA":1,   "AA":2}},
          {"MTHFR_677":   {"CC":0,   "CT":0,   "TT":1}},
          {"MTHFR_1298":  {"AA":0,   "AC":1,   "CC":2}}
        ]

In [ ]:
# Использование базы знаний в соответствии с описанием
gen_map_2 = [
          {"F2":          {"GG":0,   "GA":1,   "AA":2}},
          {"F5":          {"GG":0,   "GA":1,   "AA":2}},
          {"F7":          {"GG":0,   "GA":-1,  "AA":-2}},
          {"F13":         {"GG":0,   "GT":0,   "TT":-2}},
          {"ITGA2":       {"CC":0,   "CT":0,   "TT":2}},
          {"ITGB3":       {"TT":0,   "TC":1,   "CC":2}},
          {"PAI_1":       {"5G5G":0, "5G4G":0, "4G4G":2}},
          {"FGB":         {"GG":0,   "GA":1,   "AA":2}},
          {"MTHFR_677":   {"CC":0,   "CT":0,   "TT":2}},
          {"MTHFR_1298":  {"AA":0,   "AC":1,   "CC":2}}
        ]

df_2 = pd.DataFrame()
for m in gen_map_2:    
    [(k,v)] = m.items()
    df_2[k + '__2'] = df[k].map(v)

df_ext = pd.concat([df_ext, df_2], axis = 1)   # https://pandas.pydata.org/pandas-docs/stable/merging.html

df_2.head(3)

In [ ]:
genetic__2 = list( df_2.columns )
print(genetic__2)

features = features + [('genetic__2', genetic__2)]

#### Комбинации генетических признаков.

Группировка известных факторов либо увеличения, либо снижения риска тромбофилии - согласно открытым базам знаний лабораторий.
<br> Суммы по группам. Масштабирование вклада групп. 

In [ ]:
# Группы генетических факторов: увеличение или снижение риска тробофилии
genetic_guess = [
    {'GenRisk__1':['F2__1', 'F5__1', 'ITGA2__1', 'ITGB3__1', 'PAI_1__1', 'FGB__1', 'MTHFR_677__1', 'MTHFR_1298__1']},
    {'GenRisk__2':['F2__2', 'F5__2', 'ITGA2__2', 'ITGB3__2', 'PAI_1__2', 'FGB__2', 'MTHFR_677__2', 'MTHFR_1298__2']},
    {'GenProtect__1':['F7__1', 'F13__1']},
    {'GenProtect__2':['F7__2', 'F13__2']} ]

# Суммы по группам
df_gg = pd.DataFrame()
for g in genetic_guess:
    #print(g)       #print( g.items() )         #print( k,v )
    [(k,v)] = g.items()
    df_gg[k] = np.array(df_ext[v].sum(axis=1))

df_ext = pd.concat([df_ext, df_gg], axis=1)   # https://pandas.pydata.org/pandas-docs/stable/merging.html

In [ ]:
# Сгруппированные генетические признаки для использования в модели
genetic_risk__1 = ['GenRisk__1', 'GenProtect__1']
genetic_risk__2 = ['GenRisk__2', 'GenProtect__2']
genetic_risk    = ['GenRisk__1', 'GenRisk__2', 'GenProtect__1', 'GenProtect__2']

features = features + [('genetic_risk__1', genetic_risk__1)]
features = features + [('genetic_risk__2', genetic_risk__2)]
features = features + [('genetic_risk',    genetic_risk)]

In [ ]:
df_ext[genetic_risk].head(3)

#### Масштабируем генетические признаки

In [ ]:
print( df_ext[genetic_risk].describe() )

In [ ]:
df_ext['GenRisk__1_scaled']    = (df_ext['GenRisk__1'].copy()    - 4.23)  / 1.55
df_ext['GenProtect__1_scaled'] = (df_ext['GenProtect__1'].copy() - 0.81)  / 0.74

df_ext['GenRisk__2_scaled']    = (df_ext['GenRisk__2'].copy()    - 2.94)  / 1.85
df_ext['GenProtect__2_scaled'] = (df_ext['GenProtect__2'].copy() + 0.405) / 0.68

In [ ]:
genetic_risk__1_scaled = ['GenRisk__1_scaled', 'GenProtect__1_scaled']
genetic_risk__2_scaled = ['GenRisk__2_scaled', 'GenProtect__2_scaled']
genetic_risk_scaled    = ['GenRisk__1_scaled',    'GenRisk__2_scaled', 
                          'GenProtect__1_scaled', 'GenProtect__2_scaled']

# Remove last 3 elements from the list
# features = features[:-3]
#[0,1,2,3,4,5,6,7,8,9][:-3]

features = features + [('genetic_risk__1_scaled', genetic_risk__1_scaled)]
features = features + [('genetic_risk__2_scaled', genetic_risk__2_scaled)]
features = features + [('genetic_risk_scaled',    genetic_risk_scaled)]

In [ ]:
print( df_ext[genetic_risk_scaled].describe() )

### Сохраняем обработанные данные и группы колонок
#### Сохраняем данные

In [ ]:
data_features_ext = combine_features(
                       [target, relatives, 
                        sex, sex_cat, 
                        bmi, bmi_scaled,
                        symptom, 
                        cirrhosis, activity, childpugh_dummy,
                        debut_age, debut_age_scaled,
                        debut_organ,
                        genetic, genetic__1, genetic__2, 
                        genetic_dummy,
                        genetic_risk__1, genetic_risk__2, 
                        genetic_risk__1_scaled, genetic_risk__2_scaled
                       ],
                       exclude)

In [ ]:
print(data_features_ext)

#### Запись данных в файл: `pd.to_cvs`
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html
<br> https://stackoverflow.com/questions/27370046/closing-file-after-using-to-csv

In [ ]:
outfile = open(data_dir + 'Wilson_ext.csv', 'w')
df_ext[data_features_ext].to_csv(outfile, sep=';', index=False, encoding='utf-8', chunksize=1)
outfile.close()

#### Чтение данных из сохранённого файла

In [ ]:
df_ext2 = pd.read_csv(data_dir + 'Wilson_ext.csv', sep=';', encoding='utf-8') 
print(df_ext2.columns)

#### Сохраняем группы колонок
Сохраняем в формате `json`. Библиотека `simplejson`. 

Адаптировал к Python 3 утилиты `data_to_json` и `json_to_data`  из статьи [Serializing Python Data To Json - Some Edge Cases](http://robotfantastic.org/serializing-python-data-to-json-some-edge-cases.html) by Chris Wagner.

In [ ]:
# !conda install simplejson
%run data_to_json

In [ ]:
from collections import OrderedDict

In [ ]:
json_to_data(data_to_json(OrderedDict(features)))['bmi']  # ['target']

In [ ]:
features_file = open(data_dir + 'features_file.json', 'w')
with features_file:
    features_file.write( data_to_json(OrderedDict(features)) )
    # json.dump(json_format, features_file)
features_file.close()

In [ ]:
# json_format = json.dumps(features)
# json.loads(json_format, object_pairs_hook=OrderedDict)

##### Прочтём и проверим, что получилось

In [ ]:
features_file = open(data_dir + 'features_file.json', 'r')
features_loaded = json_to_data( features_file.read() )
#features_loaded = json.load(features_file, object_pairs_hook=OrderedDict)
features_file.close()

print(features_loaded)
type(features_loaded)

In [ ]:
features_loaded["target"]

In [ ]:
for (k,v) in features_loaded.items():
    print(k + ": ", v)